In [1]:
import copy
from utils import read_json_to_dict
from PetReader import pet_reader
from petreader.labels import *

INFO:PetReader:Load RelationsExtraction dataset ...




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

INFO:PetReader:Load TokenClassification dataset ...




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

## Load Results

In [14]:
token_cls_goldstandard = read_json_to_dict("data/other/token_goldstandard.json")
keywords_gold_token_cls_results = read_json_to_dict("data/results/key_words_gold/results-token-classification-key_words_gold.json")
keywords_gold_token_cls = read_json_to_dict("data/results/key_words_gold/token-classification.json")


## Debug

In [29]:
from petreader.TokenClassification import TokenClassification
from petbenchmarks.tokenclassification import TokenClassificationBenchmark
benchmark = TokenClassificationBenchmark(pet_reader.token_dataset)

gold = benchmark.GetGoldStandard()
print(gold['doc-3.2'][XOR_GATEWAY])

[['If'], ['otherwise'], ['If'], ['otherwise']]


In [22]:
print(pet_reader.token_dataset.GetXORGateways('doc-3.2'))

[[], [['If'], ['otherwise']], [], []]


## Analyze Gold Keyword Results

### a) check for documents with recall != 1

In [21]:
for doc_name, doc_dict in keywords_gold_token_cls_results.items():
    if doc_name.startswith('doc'):
        if doc_dict[XOR_GATEWAY][RECALL] != 1 and doc_dict[XOR_GATEWAY][SUPPORT] != 0:
            print(doc_name.center(100, '-'))
            print("--Text--")
            for i, line in enumerate(pet_reader.get_doc_sentences(doc_name)):
                print(i, ' '.join(line))
            print()
            print("--Results--:", doc_dict[XOR_GATEWAY])
            print()
            print("--Extracted--:", keywords_gold_token_cls[doc_name][XOR_GATEWAY])
            print()
            print("--Gold standard--:", token_cls_goldstandard[doc_name][XOR_GATEWAY])
            print()

----------------------------------------------doc-6.4-----------------------------------------------
--Text--
0 The first step is to determine contact details of potential customers .
1 This can be achieved in several ways .
2 Sometimes , we buy details for cold calls , sometimes , our marketing staff participates in exhibitions and sometimes , you just happen to know somebody , who is interested in the product .
3 Then we start calling the customer .
4 That is done by the call center staff .
5 They are determining the contact person and the budget which would be available for the project .
6 Of course , asking the customer whether he is generally interested is also important .
7 If this is not the case , we leave him alone , except if the potential project budget is huge .
8 Then the head of development personally tries to acquire the customer .
9 If the customer is interested in the end , the next step is a detailed online presentation .
10 It is given either by a sales representativ

## b) Analyze False Positives

In [69]:
def analyze_keyword_stats(gateway_type: str, doc_names=[]):
    keyword_stats = {}
    empty_stats_dict = {"TP": 0, "FP": 0, "FN": 0, "TPs": [], "FPs": [], "FNs": []}

    for doc_name, doc_dict in keywords_gold_token_cls_results.items():
        
        if doc_name.startswith('doc'):  # result dict contains as well keys for statistics            
            if not doc_names or doc_name in doc_names:  # filter for (optionally) passed list

                # remove gateways step by step if they were identified; left overs are false positives
                not_discovered_golds = [[t.lower() for t in g] for g in token_cls_goldstandard[doc_name][gateway_type].copy()]

                for extracted in keywords_gold_token_cls[doc_name][gateway_type]:
                    extracted = [t.lower() for t in extracted]
                    extracted_key = ' '.join(extracted)

                    # setup keyword dict in case it was not observed yet
                    if extracted_key not in keyword_stats:
                        keyword_stats[extracted_key] = copy.deepcopy(empty_stats_dict)

                    # 1) CHECK FOR FPs
                    if extracted not in not_discovered_golds:
                        keyword_stats[extracted_key]["FP"] += 1
                        keyword_stats[extracted_key]["FPs"].append(doc_name)

                    # 2) CHECK FOR TPs
                    else:
                        keyword_stats[extracted_key]["TP"] += 1
                        keyword_stats[extracted_key]["TPs"].append(doc_name)
                        not_discovered_golds.remove(extracted)

                # 3) FILL FNs (FPs from list not_discovered_xor_golds got removed during previous loop)
                for not_extracted in not_discovered_golds:
                    not_extracted_key = ' '.join(not_extracted)

                    # setup keyword dict in case it was not observed yet
                    if not_extracted_key not in keyword_stats:
                        keyword_stats[not_extracted_key] = copy.deepcopy(empty_stats_dict)

                    keyword_stats[not_extracted_key]["FN"] += 1
                    keyword_stats[not_extracted_key]["FNs"].append(doc_name)
    
    return keyword_stats
                
xor_keyword_stats = analyze_keyword_stats(XOR_GATEWAY)# ["doc-3.2"])
and_keyword_stats = analyze_keyword_stats(AND_GATEWAY)# ["doc-3.2"])